### Import values to the Netherlands from other 5 EU countries

In [1]:
import pandas as pd

# Updated file path
file_path = r'C:\Users\Icha\OneDrive - Delft University of Technology\02 Course\Q1\TIL Python Programming\Project\TIL Python Programming - Group 18\Visualization\Oil Price vs Import\Summarized Data.csv'

# Load the Summarized dataset into a DataFrame for Import value information
export_data = pd.read_csv(file_path, usecols=['Year, Quarter', 'Price', 'Import Germany', 'Import Belgium', 'Import France', 'Import Italy', 'Import Poland']).dropna()

# Apply border styling
styled_df = export_data.style.set_table_styles(
    [{'selector': 'th', 'props': [('border', '1px solid black')]},
     {'selector': 'td', 'props': [('border', '1px solid black')]}]
)

# Display the styled table
styled_df


,"Year, Quarter",Price,Import Germany,Import Belgium,Import France,Import Italy,Import Poland
0,2021 Q1,60.820000,26.110000,13.610000,5.390000,4.000000,3.390000
1,2021 Q2,68.830000,27.340000,15.080000,5.640000,4.520000,3.490000
2,2021 Q3,73.470000,26.680000,15.760000,5.580000,4.190000,3.390000
3,2021 Q4,79.590000,28.630000,16.880000,6.440000,4.470000,4.170000
4,2022 Q1,100.300000,28.790000,17.290000,6.430000,4.320000,4.100000
5,2022 Q2,113.540000,29.130000,18.060000,6.630000,4.650000,4.180000
6,2022 Q3,100.710000,26.160000,17.420000,6.550000,4.030000,4.040000
7,2022 Q4,88.560000,28.880000,17.560000,6.620000,4.820000,4.300000
8,2023 Q1,81.170000,30.870000,16.490000,6.250000,4.910000,4.210000
9,2023 Q2,78.320000,30.270000,17.680000,5.980000,4.900000,4.090000


In [3]:
total_imports = export_data[['Import Germany', 'Import Belgium', 'Import France', 'Import Italy', 'Import Poland']].sum()

sum_imports_df = pd.DataFrame({
    'Country': total_imports.index,
    'Total Import Value to the Netherlands': total_imports.values
})

sum_imports_df = sum_imports_df.sort_values(by='Total Import Value to the Netherlands', ascending = False)

sum_imports_df.reset_index(drop = True, inplace = True)

styled_sum_df = sum_imports_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('border', '1px solid black')]},
     {'selector': 'td', 'props': [('border', '1px solid black')]}]
)

styled_sum_df

,Country,Total Import Value to the Netherlands
0,Import Germany,340.010000
1,Import Belgium,198.490000
2,Import France,72.660000
3,Import Italy,54.290000
4,Import Poland,47.900000


In [18]:
# Visualization of total Import values from 5 EU countries to the Netherlands

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add Oil Price line first to make it appear first in the legend
fig.add_trace(
    go.Scatter(x = export_data['Year, Quarter'], y = export_data['Price'],
               mode = 'lines',  
               name = 'Oil Price'),
    secondary_y = True,
)

# Add import data lines for each country
countries = export_data.columns[2:7].tolist()  
marker_symbols = ['circle', 'square', 'diamond', 'cross', 'x']

for i, country in enumerate(countries):
    fig.add_trace(
        go.Scatter(x = export_data['Year, Quarter'], y=export_data[country],
                   mode = 'lines',  
                   name = country),
        secondary_y = False,
    )

# Update layout
fig.update_layout(
    title = 'Import Values by Country and Oil Price Over Time',  
    xaxis = dict(showgrid=True, tickangle=45),
    yaxis = dict(showgrid=True),
    xaxis_title = 'Year, Quarter',
    yaxis_title = 'Import Values (in Million USD)',  
    yaxis2_title = 'Oil Price (in USD)',
    width = 1200,  # Increase the width to make the graph wider
    height = 600,  # Adjust the height if needed
)

# Show the figure
fig.show()


In [45]:
# Automatically select only the columns that contain 'Import' for percentage difference calculation
import_columns = [col for col in df.columns if 'Import' in col]

# Extract only the 'Year, Quarter' and 'Import' columns for calculation
import_data = df[['Year, Quarter'] + import_columns]

# Set 'Year, Quarter' as the index for calculating percentage changes
import_data.set_index('Year, Quarter', inplace=True)

# Calculate quarter-to-quarter percentage change
percent_change = import_data.pct_change() * 100  # This gives the percentage change for each column
percent_change = percent_change.rename(columns=lambda x: f"{x} % Change")

# Drop any NaN values in the first row (since pct_change produces NaN for the first row)
percent_change = percent_change.dropna().reset_index()

# Apply table styling with borders and format only numeric columns to two decimal places
styled_percent_change = percent_change.style.set_table_styles(
    [{'selector': 'th', 'props': [('border', '1px solid black')]},
     {'selector': 'td', 'props': [('border', '1px solid black')]}]
).format({col: "{:.2f}" for col in percent_change.columns if percent_change[col].dtype != "object"})

# Display the styled DataFrame
styled_percent_change

,"Year, Quarter",Import Germany % Change,Import Belgium % Change,Import France % Change,Import Italy % Change,Import Poland % Change
0,2021 Q2,4.71,10.80,4.64,13.00,2.95
1,2021 Q3,-2.41,4.51,-1.06,-7.30,-2.87
2,2021 Q4,7.31,7.11,15.41,6.68,23.01
3,2022 Q1,0.56,2.43,-0.16,-3.36,-1.68
4,2022 Q2,1.18,4.45,3.11,7.64,1.95
5,2022 Q3,-10.20,-3.54,-1.21,-13.33,-3.35
6,2022 Q4,10.40,0.80,1.07,19.60,6.44
7,2023 Q1,6.89,-6.09,-5.59,1.87,-2.09
8,2023 Q2,-1.94,7.22,-4.32,-0.20,-2.85
9,2023 Q3,-6.77,-7.41,-8.86,-7.14,0.98


In [47]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a line plot for each country
fig = make_subplots(specs=[[{"secondary_y": False}]])

# Add a line for each country's import percentage difference
for col in df_qoq.columns[1:]:  # Skip the "Year, Quarter" column
    fig.add_trace(
        go.Scatter(x=df_qoq["Year, Quarter"], y=df_qoq[col], mode='lines', name=col)
    )

# Update layout
fig.update_layout(
    title="Percentage Difference in Import Values per Country",
    xaxis_title="Year, Quarter",
    yaxis_title="Percentage Difference (%)",
    template="plotly_white"
)

# Show the plot
fig.show()